# Imagenet Example

This notebook illustrates the use of the proposed in(n)vestigation methods on the ImageNet dataset for a pretrained VGG16.

# Imports

In [1]:
import warnings
warnings.simplefilter('ignore')

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline  

import imp
import keras.backend
import keras.models
import numpy as np
import os

import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Input
from keras.optimizers import RMSprop

import innvestigate
import innvestigate.utils as iutils
import innvestigate.utils.tests.networks.base
import innvestigate.utils.visualizations as ivis

eutils = imp.load_source("utils", "../utils.py")
imgnetutils = imp.load_source("utils_imagenet", "../utils_imagenet.py")

Using TensorFlow backend.


# Data

Get a few (preselected) example images from the ImageNet dataset. In order to load your choice of images from the ImageNet dataset, place them into `innvestigate/examples`.

In [3]:
# Get some example test set images.
images, label_to_class_name = eutils.get_imagenet_data()[:2]

# Build model

Load the pretrained VGG16 net with ReLUs.

In [4]:
# Parameter
netname = "vgg16"
pattern_type = "relu"

In [5]:
# Build Model

channels_first = keras.backend.image_data_format == "channels_first"

tmp = getattr(innvestigate.applications.imagenet, netname)
net = tmp(load_weights=True, load_patterns=pattern_type)
model = keras.models.Model(inputs=net["in"], outputs=net["out"])
model.compile(optimizer="adam", loss="categorical_crossentropy")
modelp = keras.models.Model(inputs=net["in"], outputs=net["sm_out"])
modelp.compile(optimizer="adam", loss="categorical_crossentropy")

OSError: Failed to interpret file './imagenet_224_vgg_16.pattern_file.A_only.npz' as a pickle

# Analysis

Use several methods to in(n)vestigate and visualize the learned classifier on the example images.

Hereby the analyse method containing name, optional parameter, postprocessing information and a title for the final visualisation is passed into the framework by a n-tuple. The available methods are subdivided into three groups depending on their principal approach: gradient-based, pattern-based and relevance-based investigation methods. For a full list of methods please refer to the script in `innvestigate/innvestigate/analyzer/__init__.py`.

In [ ]:
color_conversion = "BGRtoRGB" if net["color_coding"] == "BGR" else None

In [7]:
# Create list of in(n)vestigation methods we want to apply to the trained model, if applicable specify optional 
# parameters

methods = [
    # NAME             POSTPROCESSING     TITLE

    # Show input.
    ("input",                 {},                            imgnetutils.image,   "Input"),

    # Function
    ("gradient",              {},                            imgnetutils.graymap, "Gradient"),
    ("smoothgrad",            {"noise_scale": 50},           imgnetutils.graymap, "SmoothGrad"),
    ("integrated_gradients",  {},                            imgnetutils.graymap, ("Integrated", "Gradients")),

    # Signal
    ("deconvnet",             {},                            imgnetutils.bk_proj, "Deconvnet"),
    ("guided_backprop",       {},                            imgnetutils.bk_proj, ("Guided", "Backprop"),),
    ("pattern.net",           {"patterns": net["patterns"]}, imgnetutils.bk_proj, "PatterNet"),

    # Interaction
    ("pattern.attribution",   {"patterns": net["patterns"]}, imgnetutils.heatmap, "Pattern", "Attribution"),
    ("lrp.z_baseline",        {},                            imgnetutils.heatmap, "LRP-Z"),
]

NameError: name 'net' is not defined

In [ ]:
# Create analyzers.
analyzers = []
print('Create analyzer for ', flush=True)
for method in methods:
    print('{} '.format(''.join(method[3])), flush=True)
    analyzers.append(innvestigate.create_analyzer(method[0], 
                                                  model, 
                                                  **method[1]))

# Apply analyzers to trained VGG model.
analysis = np.zeros([len(images), len(analyzers), 224, 224, 3])
text = []
for i, (image, y) in enumerate(images):
    image = image[None, :, :, :]
    # Predict label.
    x = preprocess(image)
    presm = model.predict_on_batch(x)[0]
    prob = modelp.predict_on_batch(x)[0]
    y_hat = prob.argmax()

   # Save label, pre-softmax activation, probabilistic output and predicted label for plots
    text.append(("%s" %label_to_class_name[y], "%.2f" %presm.max(), 
                 "%.2f" % prob.max(), "%s" %label_to_class_name[y_hat]))
    
    for aidx, analyzer in enumerate(analyzers):
        # Measure execution time
        t_start = time.time()
        print('{} '.format(''.join(methods[aidx][-1])), end='', flush=True)
        
        is_input_analyzer = methods[aidx][0] == "input"
        # Analyze.
        a = analyzer.analyze(image if is_input_analyzer else x)
        
        t_elapsed = time.time() - t_start
        print('({:.4f}s) '.format(t_elapsed), end='', flush=True)
        
        # Postprocess.
        if not np.all(np.isfinite(a)):
            print("Image %i, analysis of %s not finite: nan %s inf %s" %
                  (i, methods[aidx][3],
                   np.any(np.isnan(a)), np.any(np.isinf(a))))
        if not is_input_analyzer:
            a = postprocess(a)
        a = methods[aidx][2](a)
        analysis[i, aidx] = a[0]

In [ ]:
# Plot the analysis.

grid = [[analysis[i, j] for j in range(analysis.shape[1])]
        for i in range(analysis.shape[0])]  
row_labels = text
col_labels = [''.join(method[3]) for method in methods]

eutils.plot_image_grid(grid, row_labels, col_labels,
                       filename=None,
                       row_label_offset=0,
                       col_label_offset=0,
                       is_fontsize_adaptive=False,
                       usetex=False)